# Create the grid for FreThaw1D model
    -Author: Niccolò Tubini, Stephan Gruber, Riccardo Rigon
    -License: this work is licensed under a Creative Commons Attribution-NonCommercial 4.0 International License
-----
This code allows to create a mesh for 1D PDE problem:
    - domain discretization
    - setting parameters
    - setting initial condition
    
All output data are stored in a NetCDF file.
This file is one of the input file of your simulation.


In [1]:
%load_ext autoreload
%autoreload 2

from FreThaw1D_gridcreator import*
import os
import pandas as pd

import warnings
warnings.filterwarnings('ignore')



## Define some variables:
`grid_input_file_name`: name of the grid input file (.csv) with the local file path -> /data/Grid_input/name.csv

`ic_input_file_name`: name of the initial condition input file (.csv) with the local file path -> /data/Grid_input/name.csv

`parameter_input_file_name`: name of the parameter input file (.csv) with the local file path -> /data/Grid_input/name.csv

`output_file_name`: name of the output file (.nc) with the local file path ->  /data/Grid_NetCDF/name.nc

`output_title`: string containing a title for the output, it is saved within the file

`output_summary`: string containing a description for the output, it is saved within the file (global metadata)

`output_date`: string containing the date of file creation, optionally

`output_institution`: string containing the name of whom created the file, optionally

`dz_min`: thickness of the first layer

`b` growth rate

`interp_mdoe`: string defining the type of the 1D interpolation function used to define the initial condition 

`grid_type`: string defining how to discretize the 1D domain

In [20]:
project_path = "C:\\Users\\Niccolo\\OMS\\OMS_Project_FreThaw1D"

grid_input_file_name = project_path + "/data/Grid_input/ThermalOffset_grid.csv"

ic_input_file_name = project_path + "/data/Grid_input/ThermalOffset_ic.csv"

parameter_input_file_name = project_path + "/data/Grid_input/ThermalOffset_parameter.csv"

output_file_name =   project_path + "/data/Grid_NetCDF/ThermalOffset_dzmin_005_b_1.nc"


output_title = '''
                  '''
output_summary = '''
 Input grid for the thermal offset synthetic experiment. Initial condition 270.15 [K]. 
 The domain is descritized using an exponential function dz_i = dz_min(1+b)^(i-1)
 dz_min = 0.005 [m] and b = 0.1 [-].
'''

output_date = ''

output_institution = ''

dz_min = 0.005
b = 0.1

grid_type = 'exponential' #'classical' # 'exponential'
interp_model = "linear"

## Constant grid

In [21]:
data_grid = pd.read_csv(grid_input_file_name)
print(data_grid)

data_ic = pd.read_csv(ic_input_file_name)
print(data_ic)

data_parameter = pd.read_csv(parameter_input_file_name, comment='#')
print()
print(data_parameter)

[KMAX, eta, eta_dual, space_delta, z, z_dual, control_volume]=grid1D(data_grid,dz_min,b,grid_type)

ic = set_initial_condition(data_ic, eta, interp_model)

[rheology_ID, parameter_ID, soil_particles_density, thermal_conductivity_soil_particles, specific_thermal_capacity_soil_particles, theta_s, theta_r,
 melting_temperature, par_1, par_2, par_3, par_4] = set_parameters(data_grid, data_parameter, KMAX, eta)

write_grid_netCDF(eta, eta_dual, z, z_dual, space_delta, control_volume, ic, rheology_ID, parameter_ID, KMAX,
                  soil_particles_density, thermal_conductivity_soil_particles, specific_thermal_capacity_soil_particles,
                  theta_s, theta_r, melting_temperature, par_1, par_2, par_3, par_4,
                  output_file_name, output_title, output_institution, output_summary, output_date, grid_input_file_name, parameter_input_file_name)

  Type  eta   K  IC  rheologyID  parameterID
0    L    0 NaN NaN         0.0          0.0
1    L  -20 NaN NaN         NaN          NaN
    eta      T0
0   0.0  270.15
1 -20.0  270.15

   spDensity  spConductivity  spSpecificHeatCapacity  thetaS  thetaR  \
0       2700             3.0                    1000    0.46     0.1   

   meltingT  par1  par2  par3  par4  
0    273.15   1.5   1.2   NaN   NaN  


***SUCCESS writing!  C:\Users\Niccolo\OMS\OMS_Project_FreThaw1D/data/Grid_NetCDF/ThermalOffset_dzmin_005_b_1.nc
